In [43]:
import cv2
import os
from os.path import join, isfile
from os import makedirs
from pathlib import Path
import xml.etree.ElementTree as ET
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as ntqdm
import random
from natsort import natsorted
import matplotlib.pyplot as plt
import pandas as pd


plt.rcParams['figure.figsize'] = [15, 10]


In [44]:
data_path = '/home/masoud/Desktop/C'
video_path = Path(data_path) / 'train'
annotation_path = Path(data_path) / 'game-status'

annotations = natsorted(list(annotation_path.glob('*.xml')), key=lambda x: x.stem)
videos = natsorted(list(video_path.glob('*')), key=lambda x: x.stem)
videos = [vid for vid in videos if vid.stem in [annot.stem for annot in annotations]]

In [45]:
pairs = list(zip(videos, annotations))

In [46]:
get_label = lambda annots, label_name: [f.get('name') for f in annots if f.find('tag').get('label') == label_name]

def get_frame_nos(annot_path, label='serving-start'):
    # Parse file
    data = ET.parse(annot_path)
    # Separate all images tags
    images = data.findall('image')
    # Filter the images with no image_tag
    images = [img for img in images if img.find('tag') is not None]
    # Get the image frame number of the ones that matches the label given like `serving-start`, ....
    frame_str = get_label(images, label)
    frame_nos = [int(f.split('_')[1]) for f in frame_str]
    return frame_nos

# Finding the annotation mistakes

In [6]:
# annot = annotations[7]

# serves = get_frame_nos(annot, 'serving-start')
# end_serves = get_frame_nos(annot, 'serving-end')
# end_games = get_frame_nos(annot, 'in-game-end')

# max_length = max(len(serves), len(end_serves), len(end_games))

# print(annot)
# print("serves: ", len(serves))
# print("end-serves: ", len(end_serves))
# print("end-games: ", len(end_games))


# all_periods = []

# print("prev_end_game | serve-start | end-serve | end_game |")
# for i in range(max_length):
#     eg = end_games[i]
    
#     next_s = serves[i+1]
#     next_es = end_serves[i+1]
#     next_eg = end_games[i+1]
    
#     print(f"{eg}           |     {next_s}     |   {next_es}     |   {next_eg}    |")
#     assert eg < next_s, f"the end_game {eg} - start_serve {next_s}"
#     assert next_s < next_es, f"The end_serve {next_es} - start serve {next_s}"
#     assert next_es < next_eg, f"the end_serve {next_es} is after end_game {next_eg}"
    

/home/masoud/Desktop/C/game-status/13.xml
serves:  226
end-serves:  226
end-games:  226
prev_end_game | serve-start | end-serve | end_game |
237           |     368     |   411     |   510    |
510           |     653     |   716     |   899    |
899           |     980     |   1031     |   1144    |
1144           |     1252     |   1320     |   1438    |
1438           |     1529     |   1618     |   1753    |
1753           |     1857     |   1928     |   2091    |
2091           |     2147     |   2214     |   2308    |
2308           |     2486     |   2523     |   2642    |
2642           |     2783     |   2832     |   2998    |
2998           |     3057     |   3132     |   3271    |
3271           |     3390     |   3432     |   3549    |
3549           |     3658     |   3716     |   3899    |
3899           |     3942     |   4036     |   4187    |
4187           |     4268     |   4336     |   4448    |
4448           |     4572     |   4620     |   4639    |
4639          

IndexError: list index out of range

# Generate Services

In [52]:
output_base_dir = '/home/masoud/Desktop/projects/volleyball_analytics/data/raw/game-status'
service_dir = Path(output_base_dir) / 'service'
game_dir = Path(output_base_dir) / 'play'
no_game_dir = Path(output_base_dir) / 'no-play'


def write_video(cap: cv2.VideoCapture, st: int, end: int, fps: int, w: int, h: int, video_length: int, output_path: Path, video: Path):
    cap.set(1, st)
    filename = f'inplay_{video.stem}_st1_{st}_end_{st+50}.mp4'
    output_file = (output_path / filename).as_posix()
    writer = cv2.VideoWriter(
        output_file, 
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (w, h)
    )
    for i in range(video_length):
        status, frame = cap.read()
        if status:
            writer.write(frame)

    writer.release()

def get_random_int_between_ab(st, end, divisions=5, video_length=50):
    diff = end - st
    interval = diff // divisions
    resutls = []
    for i in range(5):
        start_fno = random.randint(st, (st + interval - 50))
        st += interval
        results.append(start_fno)
    return results


for video, annot in pairs:
    cap = cv2.VideoCapture(video.as_posix())
    w, h, fps = [int(cap.get(i)) for i in range(3, 6)]
    
    serves = get_frame_nos(annot, 'serving-start')
    end_serves = get_frame_nos(annot, 'serving-end')
    end_games = get_frame_nos(annot, 'in-game-end')
    data = {
    'serve': serves, 'end_serve': end_serves, 'end_game': end_games
    }
    df = pd.DataFrame(data=data)

    df['next_serve'] = None
    
    for i in range(len(df)):
        if i != len(df) - 1:
            next_serve = df.at[i+1, 'serve']
            df.at[i, 'next_serve'] = next_serve
        else:
            df.at[i, 'next_serve'] = None
    
    pbar = ntqdm(total=len(df))
    pbar.set_description(f"Processing {video.name}")
    for i, row in df.iterrows():
        
        pbar.update(1)
        start_serve_fno = row['serve']
        end_serve_fno = start_inplay_fno = row['end_serve']
        end_inplay_fno = start_noplay_fno = row['end_game']
        end_noplay_fno = row['next_serve']
        
        ######################### Generate service videos #################################
        
        # length = end_serve_fno - start_serve_fno
        # cap.set(1, start_serve_fno)

        # filename = f'serve_{video.stem}_st_{start_serve_fno}_end_{end_serve_fno}.mp4'
        # output_file = (service_dir / filename).as_posix()
        # writer = cv2.VideoWriter(
        #     output_file, 
        #     cv2.VideoWriter_fourcc(*'mp4v'),
        #     fps,
        #     (w, h)
        # )
        # for i in range(length):
        #     status, frame = cap.read()
        #     if status:
        #         writer.write(frame)

        # writer.release()

        ######################### Generate in-play videos #################################

        length = end_inplay_fno - start_inplay_fno 
        print(length)
        if length < 50:
            continue
        elif 50 <= length <= 120:
            st = random.randint(start_inplay_fno, end_inplay_fno - 50)
            write_video(
                cap=cap, 
                st=st, 
                end=st+50, 
                fps=fps, 
                w=w, 
                h=h, 
                video_length=50, 
                output_path=game_dir, 
                video=video
            )

        elif 120 < length <= 500:
            # create 2 files : 1 from beginning, 1 from the ending part
            st = start_inplay_fno
            write_video(
                cap=cap, 
                st=st, 
                end=st+50, 
                fps=fps, 
                w=w, 
                h=h, 
                video_length=50, 
                output_path=game_dir, 
                video=video
            )

            # 2
            end = end_inplay_fno
            st = end_inplay_fno - 55

            write_video(
                cap=cap, 
                st=st, 
                end=st+50, 
                fps=fps, 
                w=w, 
                h=h, 
                video_length=50, 
                output_path=game_dir, 
                video=video
            )

        # elif length > 500:
        #     st = start_inplay_fno
        #     end = end_inplay_fno
            
        #     write_video(
        #         cap=cap,
        #         st=st,
        #         end=st+50,
        #         fps=fps,
        #         w=w,
        #         h=h,
        #         video_length=50,
        #         output_path=game_dir, 
        #         video=video
        #     )

        #     fnos = get_random_int_between_ab(st, end)

        #     for fno in fnos:
        #         write_video(
        #             cap=cap,
        #             st=fno,
        #             end=st+50,
        #             fps=fps,
        #             w=w,
        #             h=h,
        #             video_length=50,
        #             output_path=game_dir, 
        #             video=video
        #         )


    

  0%|          | 0/20 [00:00<?, ?it/s]

129
120
115
108
108
116
117
110
98
179
274
335
159
248
160
126
304
158
264
247


  0%|          | 0/12 [00:00<?, ?it/s]

567
217
303
356
388
503
933
177
747
478
647
414


  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/42 [00:00<?, ?it/s]

43
123
406
134
282
481
53
-761
-679
-670
-3618
-812
-837
-567
-692
-756
-798
-708
-741
-699
-872
-767
-624
-2411
87
511
452
30
88
115
237
26
134
368
50
261
548
110
113
269
37
166


  0%|          | 0/134 [00:00<?, ?it/s]

333
130
170
147
47
180
128
736
417
117
142
110
27
360
162
95
29
509
117


KeyboardInterrupt: 

In [55]:
video, annot = pairs[3]
serves = get_frame_nos(annot, 'serving-start')
end_serves = get_frame_nos(annot, 'serving-end')
end_games = get_frame_nos(annot, 'in-game-end')
data = {
'serve': serves, 'end_serve': end_serves, 'end_game': end_games
}
df = pd.DataFrame(data=data)




In [57]:
df['diff'] = df["end_serve"] - df["serve"]
df['diff2'] = df['end_game'] - df['end_serve']
df

,serve,end_serve,end_game,diff,diff2
0,736,797,840,61,43
1,1361,1433,1556,72,123
2,3211,3273,3679,62,406
3,4332,4375,4509,43,134
4,5264,5297,5579,33,282
5,6255,6293,6774,38,481
6,9917,9963,10016,46,53
7,11447,11509,10748,62,-761
8,12260,12314,11635,54,-679
9,12965,13024,12354,59,-670


# Generate In-Game periods

- find the interval of end2_game and end1_game
- if it's between 50 and 120 get 1 video from it.
- if it's more than 120, then get 2 videos. one from beginning, the other from end of it.
- if it's more than 500, take 5 videos. each from 100 frame intervals.

## Next Steps

- Train a sample model and try splitting videos based on it

 

(2037, 2629, 2219)
(2629, 3221, 2865)
(3221, 3813, 3578)
(3813, 4405, 3829)
(4405, 4997, 4852)


In [46]:
serves[224]

68193

In [21]:
(df['end_serve'] - df['serve']).max()


94

In [20]:
(df['end_serve'] - df['serve']).min()

32

In [23]:
# print(end1_frame_no)
print(end2_frame_no - end1_frame_no)

117


In [26]:
S = service_dir.as_posix()

In [29]:
!ls $S | wc -l

680


In [39]:
serves

[70,
 368,
 653,
 980,
 1252,
 1529,
 1857,
 2147,
 2486,
 2783,
 3057,
 3390,
 3658,
 3942,
 4268,
 4572,
 4875,
 5154,
 5435,
 5758,
 6052,
 6388,
 6681,
 6987,
 7259,
 7565,
 7871,
 8160,
 8439,
 8747,
 9085,
 9365,
 9668,
 9948,
 10271,
 10576,
 10853,
 11166,
 11449,
 11753,
 12079,
 12376,
 12665,
 12979,
 13245,
 13590,
 13842,
 14149,
 14456,
 14781,
 15097,
 15403,
 15671,
 15982,
 16262,
 16564,
 16849,
 17165,
 17474,
 17749,
 18057,
 18339,
 18637,
 18958,
 19266,
 19586,
 19830,
 20160,
 20459,
 20755,
 21064,
 21338,
 21671,
 21976,
 22254,
 22574,
 22872,
 23158,
 23449,
 23742,
 24042,
 24395,
 24639,
 24955,
 25261,
 25554,
 25856,
 26147,
 26464,
 26741,
 27041,
 27338,
 27693,
 27977,
 28266,
 28577,
 28827,
 29151,
 29455,
 29760,
 30044,
 30363,
 30681,
 30967,
 31281,
 31579,
 31840,
 32152,
 32485,
 32750,
 33066,
 33369,
 33658,
 33941,
 34236,
 34541,
 34855,
 35180,
 35465,
 35763,
 36070,
 36369,
 36651,
 36962,
 37292,
 37556,
 37864,
 38151,
 38447,
 38746,

In [41]:
df

,serve,end_serve,end_game
0,70,132,237
1,368,411,510
2,653,716,899
3,980,1031,1144
4,1252,1320,1438
...,...,...,...
221,66938,67017,67193
222,67247,67318,67491
223,67548,67614,67634
224,67857,67926,68027
